# Basic Datashader Example

In [ ]:
import traitlets as T

import ipywidgets as W
from ipyradiant import DatashaderVisualizer, LayoutSelector
from rdflib import Graph
from requests_cache import CachedSession

In [ ]:
session = CachedSession(".cache")

Here we are creating a DatashaderExample class (subclass of ipywidgets VBox) to display
the datashader image. We also attach a VisSelector widget (found in tools.py), which is
used to change the layout of the graph via traitlets linking.

In [ ]:
class DatashaderExample(W.VBox):
    ds = T.Instance(DatashaderVisualizer)
    vs = T.Instance(LayoutSelector)
    graph = T.Instance(Graph, allow_none=True)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        T.link((self, "graph"), (self.ds, "graph"))
        T.link((self, "ds"), (self.vs, "vis"))
        self.children = [self.vs, self.ds]

    @T.default("graph")
    def _make_default_graph(self):
        rdf_graph = Graph()
        full_graph = rdf_graph.parse(
            data=session.get(
                "https://raw.githubusercontent.com/schemaorg/schemaorg/v9.0-release/data/releases/8.0/schema.nt"
            ).text,
            format="nt",
        )
        return full_graph

    @T.default("ds")
    def _make_default_ds(self):
        return DatashaderVisualizer()

    @T.default("vs")
    def _make_default_vs(self):
        return LayoutSelector()

The widget is displayed below, with the VisSelector included.

In [ ]:
ds_example = DatashaderExample()
ds_example

With the DatashaderVis class, we are able to access the selected_nodes attribute. This
is updated every time a node is either clicked on or if a box is drawn over a group of
nodes. The value is automatically updated with the URIRefs of the nodes selected.

In [ ]:
ds_example.ds.selected_nodes